이 코드는 2021년 12월 14일에 tensorflow 2.7 버전으로 마지막으로 테스트 되었습니다.  

이 코드는 위키독스 '딥 러닝을 이용한 자연어 처리 입문'의 seq2seq 튜토리얼입니다.

링크 : https://wikidocs.net/24996

In [1]:
import os
import urllib3
import zipfile
import shutil

import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

tf.__version__

'2.9.3'

In [2]:
http = urllib3.PoolManager()
url ='http://www.manythings.org/anki/fra-eng.zip'
filename = 'fra-eng.zip'
path = os.getcwd()
zipfilename = os.path.join(path, filename)
with http.request('GET', url, preload_content=False) as r, open(zipfilename, 'wb') as out_file:       
    shutil.copyfileobj(r, out_file)

with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
    zip_ref.extractall(path)

In [3]:
lines = pd.read_csv('fra.txt', names=['src', 'tar', 'lic'], sep='\t')
del lines['lic']
print('전체 샘플의 개수 :',len(lines))

전체 샘플의 개수 : 217975


In [4]:
lines = lines.loc[:, 'src':'tar']
lines = lines[0:60000] # 6만개만 저장
lines.sample(10)

,src,tar
17744,We need a medic.,Il nous faut un médecin.
3533,I'm prudent.,Je suis prudent.
43801,Who did you go with?,Avec qui êtes-vous allé ?
9088,They're crazy.,Elles sont folles.
59859,You forgot the spoons.,Tu as oublié les cuillères.
46871,I know it's not fair.,Je sais que ce n'est pas juste.
11715,Is Tom working?,Est-ce que Tom travaille ?
7660,I have doubts.,J'ai des doutes.
35008,There is some wind.,Il y a un peu de vent.
37153,You'd better leave.,Mieux vaut que vous partiez.


In [5]:
lines.tar = lines.tar.apply(lambda x : '\t ' + x + ' \n')
lines.sample(10)

,src,tar
57709,The moon has come out.,\t La lune est apparue. \n
51488,Where are the horses?,\t Où sont les chevaux ? \n
1811,Do I stink?,\t Est-ce que je pue ? \n
58725,Tom's cat is adorable.,\t Le chat de Tom est adorable. \n
6093,Thanks a lot.,\t Merci bien. \n
26092,I like your style.,\t J'aime votre style. \n
40967,It's useless to try.,\t C'est inutile d'essayer. \n
54072,I acted on his advice.,\t J'ai agi conformément à son conseil. \n
50112,This is a great idea.,\t C'est une idée merveilleuse. \n
5472,I'm not dead.,\t Je ne suis pas morte. \n


In [6]:
# 글자 집합 구축
src_vocab = set()
for line in lines.src:  # 1줄씩 읽음
    for char in line:   # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab = set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

In [7]:
src_vocab_size = len(src_vocab) + 1
tar_vocab_size = len(tar_vocab) + 1
print('source 문장의 char 집합 :', src_vocab_size)
print('target 문장의 char 집합 :', tar_vocab_size)

source 문장의 char 집합 : 80
target 문장의 char 집합 : 103


In [8]:
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))
print(src_vocab[45:75])
print(tar_vocab[45:75])

['W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
['T', 'U', 'V', 'W', 'X', 'Y', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x']


In [9]:
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{' ': 1, '!': 2, '"': 3, '$': 4, '%': 5, '&': 6, "'": 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, '?': 23, 'A': 24, 'B': 25, 'C': 26, 'D': 27, 'E': 28, 'F': 29, 'G': 30, 'H': 31, 'I': 32, 'J': 33, 'K': 34, 'L': 35, 'M': 36, 'N': 37, 'O': 38, 'P': 39, 'Q': 40, 'R': 41, 'S': 42, 'T': 43, 'U': 44, 'V': 45, 'W': 46, 'X': 47, 'Y': 48, 'Z': 49, 'a': 50, 'b': 51, 'c': 52, 'd': 53, 'e': 54, 'f': 55, 'g': 56, 'h': 57, 'i': 58, 'j': 59, 'k': 60, 'l': 61, 'm': 62, 'n': 63, 'o': 64, 'p': 65, 'q': 66, 'r': 67, 's': 68, 't': 69, 'u': 70, 'v': 71, 'w': 72, 'x': 73, 'y': 74, 'z': 75, '°': 76, 'é': 77, '’': 78, '€': 79}
{'\t': 1, '\n': 2, ' ': 3, '!': 4, '"': 5, '$': 6, '%': 7, '&': 8, "'": 9, '(': 10, ')': 11, ',': 12, '-': 13, '.': 14, '0': 15, '1': 16, '2': 17, '3': 18, '4': 19, '5': 20, '6': 21, '7': 22, '8': 23, '9': 24, ':': 25, '?': 26, 'A': 27, 'B': 28, 'C': 29, 'D': 30, 'E': 31, 'F': 32, 'G': 33, 'H': 3

In [10]:
lines.src[:5]

0    Go.
1    Go.
2    Go.
3    Go.
4    Hi.
Name: src, dtype: object

In [11]:
encoder_input = []

# 1개의 문장
for line in lines.src:
    encoded_line = []
    # 각 줄에서 1개의 char
    for char in line:
        # 각 char을 정수로 변환
        encoded_line.append(src_to_index[char])
    encoder_input.append(encoded_line)

print('source 문장의 정수 인코딩 :', encoder_input[:5])

source 문장의 정수 인코딩 : [[30, 64, 10], [30, 64, 10], [30, 64, 10], [30, 64, 10], [31, 58, 10]]


In [12]:
lines.tar[:5]

0          \t Va ! \n
1       \t Marche. \n
2    \t En route ! \n
3       \t Bouge ! \n
4       \t Salut ! \n
Name: tar, dtype: object

In [13]:
decoder_input = []

for line in lines.tar:
    encoded_line = []
    for char in line:
        encoded_line.append(tar_to_index[char])
    decoder_input.append(encoded_line)

print('target 문장의 정수 인코딩 :', decoder_input[:5])

target 문장의 정수 인코딩 : [[1, 3, 48, 52, 3, 4, 3, 2], [1, 3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [1, 3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [1, 3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [1, 3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [14]:
decoder_target = []

for line in lines.tar:
    timestep = 0
    encoded_line = []
    for char in line:
        if timestep > 0:
            encoded_line.append(tar_to_index[char])
        timestep = timestep + 1
    decoder_target.append(encoded_line)
    
print('target 문장 레이블의 정수 인코딩 :', decoder_target[:5])

target 문장 레이블의 정수 인코딩 : [[3, 48, 52, 3, 4, 3, 2], [3, 39, 52, 69, 54, 59, 56, 14, 3, 2], [3, 31, 65, 3, 69, 66, 72, 71, 56, 3, 4, 3, 2], [3, 28, 66, 72, 58, 56, 3, 4, 3, 2], [3, 45, 52, 63, 72, 71, 3, 4, 3, 2]]


In [15]:
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print('source 문장의 최대 길이 :', max_src_len)
print('target 문장의 최대 길이 :', max_tar_len)

source 문장의 최대 길이 : 22
target 문장의 최대 길이 : 76


In [16]:
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

print('shape of encoder_input : ', encoder_input.shape)
print('shape of decoder_input : ', decoder_input.shape)
print('shape of decoder_target : ', decoder_target.shape)

shape of encoder_input :  (60000, 22)
shape of decoder_input :  (60000, 76)
shape of decoder_target :  (60000, 76)


In [17]:
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

print('shape of encoder_input : ', encoder_input.shape)
print('shape of decoder_input : ', decoder_input.shape)
print('shape of decoder_target : ', decoder_target.shape)

shape of encoder_input :  (60000, 22, 80)
shape of decoder_input :  (60000, 76, 103)
shape of decoder_target :  (60000, 76, 103)


In [18]:
import numpy as np
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

In [19]:
# src_vocab_size=80
# shape of encoder_inputs : (None, None, 80)
encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)

# encoder_outputs은 여기서는 불필요
# shape of encoder_outputs : (None, 256)
# shape of state_h : (None, 256)
# shape of state_c : (None, 256)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)

# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 은닉 상태와 셀 상태.
encoder_states = [state_h, state_c]

In [20]:
# tar_vocab_size=103
# shape of decoder_inputs : (None, None, 103)
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)

# 디코더에게 인코더의 은닉 상태, 셀 상태를 전달.
# shape of decoder_outputs : (None, None, 256)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

# shape of decoder_outputs : (None, None, 103)
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

In [21]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(
    optimizer="rmsprop",
    loss="categorical_crossentropy"
)

model.fit(
    x=[encoder_input, decoder_input],
    y=decoder_target,
    batch_size=64,
    epochs=50,
    validation_split=0.2
)

Epoch 1/50
750/750 [==============================] - 17s 20ms/step - loss: 0.7379 - val_loss: 0.6436
Epoch 2/50
750/750 [==============================] - 16s 21ms/step - loss: 0.4455 - val_loss: 0.5092
Epoch 3/50
750/750 [==============================] - 15s 20ms/step - loss: 0.3706 - val_loss: 0.4474
Epoch 4/50
750/750 [==============================] - 15s 21ms/step - loss: 0.3287 - val_loss: 0.4096
Epoch 5/50
750/750 [==============================] - 16s 21ms/step - loss: 0.3008 - val_loss: 0.3888
Epoch 6/50
750/750 [==============================] - 16s 21ms/step - loss: 0.2806 - val_loss: 0.3734
Epoch 7/50
750/750 [==============================] - 15s 20ms/step - loss: 0.2648 - val_loss: 0.3616
Epoch 8/50
750/750 [==============================] - 16s 21ms/step - loss: 0.2521 - val_loss: 0.3536
Epoch 9/50
750/750 [==============================] - 15s 20ms/step - loss: 0.2415 - val_loss: 0.3464
Epoch 10/50
750/750 [==============================] - 15s 20ms/step - loss: 0.232

In [22]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [23]:
encoder_model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 80)]        0         
                                                                 
 lstm (LSTM)                 [(None, 256),             345088    
                              (None, 256),                       
                              (None, 256)]                       
                                                                 
Total params: 345,088
Trainable params: 345,088
Non-trainable params: 0
_________________________________________________________________


In [24]:
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
# shape of decoder_inputs : (None, None, 103)
# shape of decoder_outputs : (None, None, 256)
# shape of state_h : (None, 256)
# shape of state_c : (None, 256)
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
# shape of decoder_outputs : (None, None, 103)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_softmax_layer(decoder_outputs)

decoder_model = Model(
    inputs=[decoder_inputs] + decoder_states_inputs,
    outputs=[decoder_outputs] + decoder_states
)

In [25]:
decoder_model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, 103)]  0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, 256)]        0           []                               
                                                                                                  
 lstm_1 (LSTM)                  [(None, None, 256),  368640      ['input_2[0][0]',                
                                 (None, 256),                     'input_3[0][0]',          

In [26]:
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

In [27]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
            len(decoded_sentence) > max_tar_len):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [28]:
for seq_index in [3, 50, 100, 300, 1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index:seq_index+1]
    decoded_sentence = decode_sequence(input_seq)
    
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][2:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역 문장:', decoded_sentence[1:len(decoded_sentence)-1]) # '\n'을 빼고 출력

1/1 [==============================] - 0s 10ms/step
-----------------------------------
입력 문장: Go.
정답 문장: Bouge ! 
번역 문장: Décampe ! 
1/1 [==============================] - 0s 10ms/step
-----------------------------------
입력 문장: Hello!
정답 문장: Bonjour ! 
번역 문장: Bonjour ! 
1/1 [==============================] - 0s 10ms/step
-----------------------------------
입력 문장: Got it!
정답 문장: Compris ! 
번역 문장: Compris ! 
1/1 [==============================] - 0s 10ms/step
-----------------------------------
입력 문장: Goodbye.
정답 문장: Au revoir. 
번역 문장: Casse-toi. 
1/1 [==============================] - 0s 10ms/step
-----------------------------------
입력 문장: Hands off.
정답 문장: Pas touche ! 
번역 문장: Va ! 
